## 🦜🔗 LangChain RAG Agent (RAG nur bei Bedarf)


In [1]:
from langchain_community.vectorstores.faiss import FAISS
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from helpers import llm

### In diesem Notebook schauen wir uns an, wie RAG als Tool funktioniert.

Es wird also nicht einfach auf gut Glück bei jeder Anfrage das RAG angeworfen und Dokumente mit in die Prompt gehängt. Das entscheidet unser Agent nun alleine.


#### Wir erstellen eine kleine in-memory FAISS Datenbank.


In [2]:
from langchain_openai import OpenAIEmbeddings


embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
loader = PyPDFLoader("LangChain.pdf")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=80, chunk_overlap=30)
docs = text_splitter.split_documents(documents)
db = FAISS.from_documents(docs, embeddings)

#### Wir benutzen LangChain-Magie um aus der Vektor-Datenbank ein Tool zu machen, das der Agent benutzen kann

Die Beschreibung "This is the best place to look for any information about LangChain." ist essentiell für die Funktionalität der gesamten App. Damit weiß das LLM, dass es genau dieses Tool aufrufen muss, wenn es Informationen zu LangChain braucht.


In [3]:
retriever = db.as_retriever()
tool = create_retriever_tool(
    retriever,
    "search_langchain_documentation",
    "This is the best place to look for any information about LangChain.",
)
tools = [tool]

#### Wir bauen den Agenten mit Hilfe von LangGraph prebuilt und geben ihm das Tool.

Das Prompt ziehen wir vom Hub. Es ist nicht sonderlich relevant.


In [4]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import SystemMessage, HumanMessage, BaseMessage

system_message = SystemMessage(content="Du bist ein hilfsbereiter Assistent.")
agent_executor = create_react_agent(llm(), tools, state_modifier=system_message)

In [5]:
input = {
    "messages": [HumanMessage(content="Wie nutze ich Langchain mit einem Vectorstore?")]
}

In [6]:
for event in agent_executor.stream(input, stream_mode="values"):
    message: BaseMessage = event["messages"][-1]
    message.pretty_print()

================================ Human Message =================================

Wie nutze ich Langchain mit einem Vectorstore?
================================== Ai Message ==================================
Tool Calls:
  search_langchain_documentation (call_W2nWgVMYuC1CFLSB4jSf2AMP)
 Call ID: call_W2nWgVMYuC1CFLSB4jSf2AMP
  Args:
    query: Langchain mit Vectorstore
================================= Tool Message =================================
Name: search_langchain_documentation

GettingStartedWithLangChain 20
LangChain Overview Wrap Up
WewillcontinueusingLangChainfortherestofthisbookaswell
astheLlamaIndexlibrarythatweintroduceinthenextchapter.
IcoverjustthesubsetofLangChainthatIuseinmyownprojects
in this book. I urge you to read the LangChain documentation
andtoexplorepublicLangChainchainsthatusershavewrittenon
Langchain-hub5.
5https://github.com/hwchase17/langchain-hub

GettingStartedWithLangChain 10
foragents,alibraryofagentstochoosefrom,andexamplesofend-
to-endagents.
LangCha

#### Echtes Streaming


In [7]:
from helpers import graph_agent_llm_output_streamer_events

await graph_agent_llm_output_streamer_events(agent_executor, input)

Um LangChain mit einem Vectorstore zu nutzen, folgen Sie diesen grundlegenden Schritten:

1. **Installation der erforderlichen Bibliotheken**:
   Sie müssen einige Pakete installieren, um mit Vectorstores arbeiten zu können. Führen Sie die folgenden Befehle in Ihrer Kommandozeile aus:
   ```bash
   pip install langchain chroma chromadb unstructured pdf2image pytesseract
   ```

2. **Importieren der notwendigen Module**:
   In Ihrem Python-Skript importieren Sie die erforderlichen Module:
   ```python
   from langchain.text_splitter import CharacterTextSplitter
   from langchain.vectorstores import Chroma
   from langchain.embeddings import OpenAIEmbeddings
   from langchain.document_loaders import DirectoryLoader
   from langchain import OpenAI, VectorDBQA
   ```

3. **Erstellen von Embeddings**:
   Sie müssen ein Embedding-Modell erstellen, um Textdokumente in Vektoren umzuwandeln. Beispielsweise:
   ```python
   embeddings = OpenAIEmbeddings()
   ```

4. **Laden von Dokumenten**:
   

## ✅ Aufgabe

In der Datei "LLMAll_de-DE.md" findet sich die deutsche Version der OWASP Top 10 für LLMs.
Erweitere den Agent so, dass er hier für Sicherheitsfragen nachschaut.


In [ ]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader

secloader = UnstructuredMarkdownLoader(
    "LLMAll_de-DE.md",
    mode="elements",
    strategy="fast",
)

# Load, split and index the documents
# and create a tool named "sectool" based on the retriever

tools = [sectool, tool]

system_message = SystemMessage(content="Du bist ein hilfsbereiter Assistent.")
agent_executor = create_react_agent(llm(), tools, system_message)

input = {"messages": [HumanMessage(content="Warum sind Prompt Injections gefährlich?")]}


await graph_agent_llm_output_streamer_events(agent_executor, input)